In [1]:
using LinearAlgebra
using Arpack
using Random
using Plots

gr(size=(1800,1600))

Plots.GRBackend()

In [2]:
function OracleSubspace(A,b,ε,q,k,Pi,x_true)
# approximation of the smallest eigenvalues.
eigvals,eigvecs = eigen(I - ε*A)
# Sort eigenvalues in descending order
sorted_indices = sortperm(abs.(eigvals), rev = true)
sorted_eigvecs = eigvecs[:, sorted_indices]

# Get the k largest eigenvectors
top_eigvecs = sorted_eigvecs[:,1:k]
# qr here
#invA = invA[:,:q-1]

error_arr = zeros(q)
error_arr_rich = zeros(q)

y = zeros(size(b))

for j=1:q
    y += ε*b - ε*A*y
    full_subspace = hcat(y,top_eigvecs)
    Q,R = qr(full_subspace)
    Q = Matrix(Q)

    AQ = A*Q
   # c = (Q'*AQ)\(Q'*b)
    c = AQ\b

    x_approx = Q*c

    error_arr[j] = norm(AQ*c - b)/norm(b)
    error_arr_rich[j] = norm(A*y-b)/norm(b)
end

return error_arr,error_arr_rich
end

OracleSubspace (generic function with 1 method)

In [3]:
function RandomStarting(A,b,ε,q,k,Pi,x_true)
n = size(A,1)
# Ensure that ε is small enough so that I - ε*A has
# spectral radius < 1
eigenvalues = eigvals(I - ε*A)
spectral_radius = maximum(abs.(eigenvalues))
if spectral_radius >= 1
    throw(ArgumentError("Choose a smaller ε such that spectral radius of I - εA
    is less than 1"))
end

Y = zeros(n,k+1)
Y[:,1:end-1] = Pi

error_arr = zeros(q)

for j=1:q
    Y[:,1:end-1] = (I - ε*A)*Y[:,1:end-1]
    Y[:,end] += ε*b - ε*A*Y[:,end]

    Q,R = qr(Y)

    Q = Matrix(Q) # skinny qr
    Y[:,1:end-1] = Q[:,1:end-1]
    
    AQ = A*Q
    #c = (Q'*AQ)\(Q'*b)
    c = AQ\b

    x_approx = Q*c

    error_arr[j] = norm(AQ*c-b)/norm(b)
end

return error_arr
end

RandomStarting (generic function with 1 method)

In [ ]:
Random.seed!(1)
# main
n = 10000
λ = @. 10 + (1:n)
A = randn(n,n) + diagm(λ)
b = randn(n)
ε = 0.0001
q_arr = 2000
k = 10
x_true = A\b
# random sketch matrix
Pi = randn(n,k)

p = plot()
for q=q_arr
    oracle_error,rich_error = OracleSubspace(A,b,ε,q,k,Pi,x_true)
    plot!(1:length(rich_error),rich_error,
        title="Accuracy vs q",
        yaxis=:log10,
        xlab="q",
        ylab="Accuracy",
        label="richardson only",
        linewidth=2,
        titlefontsize=30,
        guidefontsize=30,
        tickfontsize=30,
        legendfontsize=30)
    plot!(1:length(oracle_error),oracle_error,
        title="Accuracy vs q",
        yaxis=:log10,
        xlab="q",
        ylab="Accuracy",
        label="oracle",
        linewidth=2,
        titlefontsize=30,
        guidefontsize=30,
        tickfontsize=30,
        legendfontsize=30)
end

for q=q_arr
    algo_error = RandomStarting(A,b,ε,q,k,Pi,x_true)
    plot!(1:length(algo_error),algo_error,
        title="Accuracy vs q",
        yaxis=:log10,
        xlab="q",
        ylab="Accuracy",
        label="random_starting",
        linewidth=2,
        titlefontsize=30,
        guidefontsize=30,
        tickfontsize=30,
        legendfontsize=30)
end

display(p)


In [101]:
using LinearAlgebra
using Arpack
using Random
using Plots

Random.seed!(1)
gr(size=(1800,1600))

function OracleSubspace(A,b,ε,q,k,Pi,x_true)
eigvals,eigvecs = eigen(I - ε*A)

sorted_indices = sortperm(abs.(eigvals), rev = true)
sorted_eigvecs = eigvecs[:, sorted_indices]

top_eigvecs = sorted_eigvecs[:,1:k]

error_arr = zeros(q)
error_arr_rich = zeros(q)

y = zeros(size(b))

for j=1:q
    y += ε*b - ε*A*y
    full_subspace = hcat(y,top_eigvecs)
    Q,R = qr(full_subspace)
    Q = Matrix(Q)

    AQ = A*Q
   # c = (Q'*AQ)\(Q'*b)
    c = AQ\b

    x_approx = Q*c

    error_arr[j] = norm(AQ*c - b)/norm(b)
    error_arr_rich[j] = norm(A*y-b)/norm(b)
end

return error_arr,error_arr_rich
end

function RandomStarting(A,b,ε,q,k,Pi)
n = size(A,1)
eigenvals = eigvals(I - ε*A)
spectral_radius = maximum(abs.(eigenvals))
if spectral_radius >= 1
	throw(ArgumentError("Choose a smaller ε such that spectral radius of I - εA
	is less than 1"))
end

Y = zeros(n,k)
Y[:,1:end-1] = Pi

error_arr = zeros(q)

for j=1:q
	AY = A*Y
    Y -= ε*AY
    Y[:,end] += ε*b

    Q,R = qr(Y)

    Q = Matrix(Q)
    Y[:,1:end-1] = Q[:,1:end-1]
    
    AQ = A*Q
    c = (Q'*AQ)\(Q'*b)

    error_arr[j] = norm(AQ*c-b)/norm(b)
end

return error_arr
end

# main
n = 10000
λ = @. 10 + (1:n)
A = randn(n,n) + diagm(λ)
b = randn(n)
ε = 0.0001
q_arr = 200
k = 20
x_true = A\b
Pi = randn(n,k-1)

#p = plot()
#for q=q_arr
#    oracle_error = OracleSubspace(A,b,ε,q,k,Pi,x_true)
#    plot!(1:length(oracle_error),oracle_error,
#        title="Accuracy vs q",
#        yaxis=:log10,
#        xlab="q",
#        ylab="Accuracy",
#        label=q,
#        linewidth=2,
#        titlefontsize=30,
#        guidefontsize=30,
#        tickfontsize=30)
#end

#display(p)

p = plot()
for q=q_arr
    algo_error = RandomStarting(A,b,ε,q,k,Pi)
    plot!(1:length(algo_error),algo_error,
        title="Accuracy vs q",
        yaxis=:log10,
        xlab="q",
        ylab="Accuracy",
        label=q,
        linewidth=2,
        titlefontsize=30,
        guidefontsize=30,
        tickfontsize=30)
end

display(p)

InterruptException: InterruptException: